In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import seaborn as sns

In [3]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import nltk
import string

In [4]:
train=pd.read_csv("train_2kmZucJ.csv")
test=pd.read_csv("test_oJQbWVk.csv")

In [5]:
combi=train.append(test,ignore_index=True)

In [7]:
train.shape

(7920, 3)

In [8]:
combi.shape

(9873, 3)

In [10]:
combi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9873 entries, 0 to 9872
Data columns (total 3 columns):
id       9873 non-null int64
label    7920 non-null float64
tweet    9873 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 231.5+ KB


In [14]:
#combi.groupby('label').describe()
combi.groupby('label').describe()

id                                                                 
        count         mean          std  min      25%     50%      75%     max
label                                                                         
0.0    5894.0  3954.497285  2290.734085  1.0  1971.25  3958.5  5927.50  7920.0
1.0    2026.0  3977.962981  2274.419827  5.0  2019.50  3968.5  5966.75  7914.0

In [15]:
combi['length']=combi['tweet'].apply(len)

In [16]:
combi.head(2)

,id,label,tweet,length
0,1,0.0,#fingerprint #Pregnancy Test https://goo.gl/h1...,128
1,2,0.0,Finally a transparant silicon case ^^ Thanks t...,131


In [17]:
from nltk.corpus import stopwords


In [19]:
def text_process(mess):
    nopunc=[char for char in mess if char not in string.punctuation]
    nopunc=''.join(nopunc)
    return [word.lower() for word in nopunc.split()  if word.lower() not in stopwords.words('english')]

In [20]:
combi['tweet'].apply(text_process)

0       [fingerprint, pregnancy, test, httpsgooglh1mfq...
1       [finally, transparant, silicon, case, thanks, ...
2       [love, would, go, talk, makememories, unplug, ...
3       [im, wired, know, im, george, made, way, iphon...
4       [amazing, service, apple, wont, even, talk, qu...
5       [iphone, software, update, fucked, phone, big,...
6       [happy, us, instapic, instadaily, us, sony, xp...
7       [new, type, c, charger, cable, uk, httpwwwebay...
8       [bout, go, shopping, listening, music, iphone,...
9       [photo, fun, selfie, pool, water, sony, camera...
10      [hey, apple, make, new, ipod, dont, make, new,...
11      [ha, heavy, machinery, need, apple, really, dr...
12      [contemplating, giving, iphone, bandwagon, sim...
13      [made, another, crazy, purchase, lol, theory, ...
14      [shaqlockholmes, samlouise1991, battery, painf...
15      [deepellum, towards, downtown, dallas, bigd, s...
16      [like, share, want, 3d, phone, case, iphone, i...
17      [go, c

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer=text_process).fit(combi['tweet'])
print(len(bow_transformer.vocabulary_))

27276


In [24]:
messages_bow = bow_transformer.transform(combi['tweet'])

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
"""
train_bow = messages_bow[:7920,:]
test_bow = messages_bow[7920:,:]

xtrain, xtest, ytrain, ytest = train_test_split(train_bow, train['label'],random_state=42,test_size=0.3)
"""
train_bow=messages_bow[:7920,:]
test_bow=messages_bow[7920:,:]
xtrain,xtest,ytrain,ytest=train_test_split(train_bow,train['label'],random_state=42,test_size=0.3)

In [26]:
"""
lreg = LogisticRegression()
lreg.fit(xtrain, ytrain) 

prediction = lreg.predict_proba(xtest) 
prediction_int = prediction[:,1] >= 0.3 
prediction_int = prediction_int.astype(np.int)

f1_score(ytest, prediction_int)
"""
model=LogisticRegression()
model.fit(xtrain,ytrain)
prediction=model.predict_proba(xtest)
prediction_int=prediction[:,1]>=0.3
prediction_int=prediction_int.astype(np.int)
f1_score(ytest,prediction_int)

0.7954055994256999

In [28]:
"""
test_pred = lreg.predict_proba(test_bow)
test_pred_int = test_pred[:,1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)
test['label'] = test_pred_int
submission = test[['id','label']]
submission.to_csv('face_log_42.csv', index=False)
"""
test_pred=model.predict_proba(test_bow)
test_pred_int=test_pred[:,1]>=0.3
test_pred_int=test_pred_int.astype(np.int)
test['label']=test_pred_int
submission=test[['id','label']]
submission.to_csv('face_log_43.csv',index=False)

In [29]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow)

In [30]:
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

(9873, 27276)


In [32]:
train_tfidf = messages_tfidf[:7920,:]
test_tfidf = messages_tfidf[7920:,:]

xtrain, xtest, ytrain, ytest = train_test_split(train_tfidf, train['label'],random_state=42,test_size=0.3)

In [33]:
model.fit(xtrain,ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [34]:
prediction = model.predict_proba(xtest) 
prediction_int = prediction[:,1] >= 0.3 
prediction_int = prediction_int.astype(np.int)

f1_score(ytest, prediction_int)

0.8031953522149601

In [36]:
test_pred = model.predict_proba(test_bow)
test_pred_int = test_pred[:,1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)
test['label'] = test_pred_int
submission = test[['id','label']]
submission.to_csv('face_log_tfidf_43.csv', index=False)

In [39]:
xtrain, xtest, ytrain, ytest = train_test_split(train_bow, train['label'],test_size=0.3)

In [40]:
model.fit(xtrain, ytrain) 

prediction = model.predict_proba(xtest) 
prediction_int = prediction[:,1] >= 0.3 
prediction_int = prediction_int.astype(np.int)

f1_score(ytest, prediction_int)

0.7812041116005874

In [41]:
test_pred = model.predict_proba(test_bow)
test_pred_int = test_pred[:,1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)
test['label'] = test_pred_int
submission = test[['id','label']]
submission.to_csv('face_log.csv', index=False)

In [42]:
xtrain, xtest, ytrain, ytest = train_test_split(train_tfidf, train['label'],test_size=0.3)

In [43]:
model.fit(xtrain,ytrain)

prediction = model.predict_proba(xtest) 
prediction_int = prediction[:,1] >= 0.3 
prediction_int = prediction_int.astype(np.int)

f1_score(ytest, prediction_int)

0.7946428571428572

In [45]:
test_pred = model.predict_proba(test_bow)
test_pred_int = test_pred[:,1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)
test['label'] = test_pred_int
submission = test[['id','label']]
submission.to_csv('face_log_tfidf.csv', index=False)